# tf.function으로 성능 향상하기

* https://www.tensorflow.org/guide/function?hl=ko

In [1]:
import tensorflow as tf

In [2]:
import traceback
import contextlib

@contextlib.contextmanager
def assert_raises(error_class):
    try:
        yield
    except error_class as e:
        print('기대하는 예외 발생 \n  {}:'.format(error_class))
        traceback.print_exc(limit=2)
    except Exception as e:
        raise e
    else:
        raise Exception('{}를 기대했지만 아무런 에러도 발생되지 않았습니다!'.format(
            error_class))

# 트레이싱과 다형성

In [3]:
# 다형성을 지원하는 python에서의 tf.function은 인자의 형 변환별로 
# 별도의 concrete function을 생성하여 트래이싱한다.

@tf.function
def double(a):
    print("Tracing: ", a)
    return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing:  Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing:  Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing:  Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



In [4]:
# TensorSpec으로 구체적인 입력 형태를 정의해주면 다형성을 지원하지 않고 
# 잘못된 형 입력에 대해 InvalidArgumentError 을 발생시킬 수 있다. 

double_strings = double.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.string))

double_strings(tf.constant("a"))

with assert_raises(tf.errors.InvalidArgumentError):
    double_strings(tf.constant(1.0))

Tracing:  Tensor("a:0", dtype=string)
기대하는 예외 발생 
  <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>:


Traceback (most recent call last):
  File "<ipython-input-2-36eb58ea7fc5>", line 7, in assert_raises
    yield
  File "<ipython-input-4-167cb9f63d43>", line 9, in <module>
    double_strings(tf.constant(1.0))
tensorflow.python.framework.errors_impl.InvalidArgumentError: cannot compute __inference_double_27 as input #0(zero-based) was expected to be a string tensor but is a float tensor [Op:__inference_double_27]


In [5]:
# 형(type) 뿐만 아니라  dimension 도 따질 수 있다.

# expect int32 1D tensor
@tf.function(input_signature=(tf.TensorSpec(shape=[None], dtype=tf.int32),))
def next_collatz(x):
    print("Tracing:", x)
    return tf.where(x % 2 == 0, x // 2, 3*x+1)

print(next_collatz(tf.constant([1,2])))

with assert_raises(ValueError):
    next_collatz(tf.constant([[1,2], [3,4]]))

Tracing: Tensor("x:0", shape=(None,), dtype=int32)
tf.Tensor([4 1], shape=(2,), dtype=int32)
기대하는 예외 발생 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-2-36eb58ea7fc5>", line 7, in assert_raises
    yield
  File "<ipython-input-5-626e12206e4c>", line 12, in <module>
    next_collatz(tf.constant([[1,2], [3,4]]))
ValueError: Python inputs incompatible with input_signature:
  inputs: (
    tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32))
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.int32, name=None))


# 파이썬 매개변수 vs 텐서 매개변수

* 매개변수에 파이선이 사용되고, 그 변수값이 바뀌면 함수가 그래프가 다시 트래이싱된다. 
* 동일한 동작을 하는 노드들이 불필요하게 add되기 때문에 매우 비효율적이다. 
* 이것이 싫으면 Tensor로 매개변수를 사용하면 한번만 트래이싱된다.

In [7]:
def train_one_step():
    pass

@tf.function
def train(num_steps):
    print('Tracing num_steps = {}'.format(num_steps))
    for _ in tf.range(num_steps):
        train_one_step()
        
train(num_steps=10)
train(num_steps=20)

Tracing num_steps = 10
Tracing num_steps = 20


In [8]:
train(num_steps=tf.constant(10))
train(num_steps=tf.constant(20))

Tracing num_steps = Tensor("num_steps:0", shape=(), dtype=int32)


In [13]:
@tf.function
def f(x):
    print("트레이싱", x)
    tf.print("실행", x)
    
f(1)
f(1)
f(2)

트레이싱 1
실행 1
실행 1
트레이싱 2
실행 2


In [16]:
def side_effect(x):
    print("Python side effect")
    pass

@tf.function
def f(x):
    tf.py_function(side_effect, inp=[x], Tout=[])
    
    
f(1)
f(1)
f(1)

Python side effect
Python side effect
Python side effect


# 파이썬 상태 주의하기

* 상태를 갖는 iterator 사용은 side-effect 이므로 트래이싱시에 단 한번만 사용되서 freeze된다. 
* 즉 next의 효과를 전혀 거둘 수 없다

In [19]:
@tf.function
def buggy_consume_next(iterator):
    a = next(iterator)
    tf.print("external iterator value: ", a)
    
iterator = iter([0, 1, 2, 3])
buggy_consume_next(iterator)
buggy_consume_next(iterator)
buggy_consume_next(iterator)

external iterator value:  0
external iterator value:  0
external iterator value:  0


# 함수 호출시마다 변수 생성 사용의 경우

* eager mode에서는 새로운 변수가 재생성
* tf.function에서는 단 한번만 생성되서 사용 => 반복 call때 buggy code가 우려되서 ValueError 발생

In [23]:
@tf.function
def f(x):
    v = tf.Variable(1.0)
    v.assign_add(x)
    return v
with assert_raises(ValueError):
    f(1)

기대하는 예외 발생 
  <class 'ValueError'>:


Traceback (most recent call last):
  File "<ipython-input-2-36eb58ea7fc5>", line 7, in assert_raises
    yield
  File "<ipython-input-23-99dd4d61e1aa>", line 7, in <module>
    f(1)
ValueError: in user code:

    <ipython-input-20-7ba303a92b55>:3 f  *
        v = tf.Variable(1.0)
    /home/hoondori/anaconda3/envs/ai/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:262 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    /home/hoondori/anaconda3/envs/ai/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:256 _variable_v2_call
        shape=shape)
    /home/hoondori/anaconda3/envs/ai/lib/python3.6/site-packages/tensorflow/python/ops/variables.py:67 getter
        return captured_getter(captured_previous, **kwargs)
    /home/hoondori/anaconda3/envs/ai/lib/python3.6/site-packages/tensorflow/python/eager/def_function.py:702 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated functio

# 오토그래프 변환

In [24]:
# 간단한 루프

@tf.function
def f(x):
    while tf.reduce_sum(x) > 1:
        tf.print(x)
        x = tf.tanh(x)
    return x

f(tf.random.uniform([5]))

[0.902372241 0.00800454617 0.318633199 0.170491219 0.280462742]
[0.717451036 0.0080043748 0.308270544 0.16885829 0.273333311]
[0.615327954 0.00800420344 0.298862934 0.167271465 0.266723722]
[0.547866702 0.00800403208 0.290271699 0.165728644 0.260573596]
[0.498919666 0.00800386071 0.282384843 0.164227813 0.25483194]
[0.461267084 0.00800368935 0.275110871 0.162767112 0.249455348]
[0.431116372 0.00800351799 0.268374056 0.161344782 0.244406611]
[0.406253844 0.00800334662 0.262111247 0.159959152 0.239653617]
[0.385287195 0.00800317526 0.256269157 0.158608675 0.235168532]
[0.367290258 0.0080030039 0.250802636 0.157291889 0.230927065]


<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.35161924, 0.00800283, 0.24567299, 0.1560074 , 0.22690786],
      dtype=float32)>

In [25]:
print(tf.autograph.to_code(f.python_function))

def tf__f(x):
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:
        do_return = False
        retval_ = ag__.UndefinedReturnValue()

        def get_state():
            return (x,)

        def set_state(vars_):
            nonlocal x
            (x,) = vars_

        def loop_body():
            nonlocal x
            ag__.converted_call(ag__.ld(tf).print, (ag__.ld(x),), None, fscope)
            x = ag__.converted_call(ag__.ld(tf).tanh, (ag__.ld(x),), None, fscope)

        def loop_test():
            return (ag__.converted_call(ag__.ld(tf).reduce_sum, (ag__.ld(x),), None, fscope) > 1)
        ag__.while_stmt(loop_test, loop_body, get_state, set_state, ('x',), {})
        try:
            do_return = True
            retval_ = ag__.ld(x)
        except:
            do_return = False
            raise
        return fscope.ret(retval_, do_return)



In [27]:
@tf.function
def fizzbuzz(n):
    for i in tf.range(1, n + 1):
        print('루프 트레이싱')
        if i % 15 == 0:
            print('fizzbuzz 브랜치 트레이싱')
            tf.print('fizzbuzz')
        elif i % 3 == 0:
            print('fizz 브랜치 트레이싱')
            tf.print('fizz')
        elif i % 5 == 0:
            print('buzz 브랜치 트레이싱')
            tf.print('buzz')
        else:
            print('디폴트 브랜치 트레이싱')
            tf.print(i)

fizzbuzz(tf.constant(1))
#fizzbuzz(tf.constant(20))

루프 트레이싱
fizzbuzz 브랜치 트레이싱
fizz 브랜치 트레이싱
buzz 브랜치 트레이싱
디폴트 브랜치 트레이싱
1


# 반복문

* 텐서플로우 반복문 ops는 단 한번만 tf.Graph에 포함되는 반면에
* 순수 파이선 반복문인 경우 (다른 python data로 tf.function을 호출하게 되면) 여러번 tf.Graph에 포함되는 엄청난 부작용이 생긴다. 

In [30]:
def measure_graph_size(f, *args):
    g = f.get_concrete_function(*args).graph
    print('{}({})는 그래프에 {}개의 노드를 포함합니다'.format(
        f.__name__, ', '.join(map(str, args)), len(g.as_graph_def().node)))
    
@tf.function
def train(dataset):
    loss = tf.constant(0)
    for x, y in dataset:
        loss += tf.abs(y - x) # 의미없는 연산
    return loss    

small_data = [(1, 1)] * 1
big_data = [(1, 1)] * 2
measure_graph_size(train, small_data)
measure_graph_size(train, big_data)

measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: small_data, (tf.int32, tf.int32)))
measure_graph_size(train, tf.data.Dataset.from_generator(
    lambda: big_data, (tf.int32, tf.int32)))

train([(1, 1)])는 그래프에 5개의 노드를 포함합니다
train([(1, 1), (1, 1)])는 그래프에 8개의 노드를 포함합니다
train(<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>)는 그래프에 8개의 노드를 포함합니다
train(<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>)는 그래프에 8개의 노드를 포함합니다
